In [30]:
# from tweepy import Stream
from tweepy import OAuthHandler
import json
import jsonpickle
import pytz
import math
from pytz import timezone
from tweepy.streaming import StreamListener
from uber_rides.session import Session
from uber_rides.client import UberRidesClient

consumer_key = 'gswlPdHuI7e0qWBAT5OavoH1w'
consumer_secret = 'rvWMe0ldojIsYSUISbcqKxFodIl0VeMGSLf8HOWUcSL6qOywgK'
access_token = '27651816-os3dMfworD2o430p66jdr668XVzvAOnz16YZgAtfj'
access_secret = 'cE8cV46N6qrKzWfuIVogYPaeM003hd2VH6Y38S4T39YGs'
 
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
keyword_list = ['traffic','accident',] #track list
#loc= [-122.75,36.8,-121.75,37.8]
loc=[-80.9138,35.1386,-80.7380,35.3539]

uber_session = Session(server_token='-l3XEhlZRPidhD3pbwMAT44BAn6W-9ZdWqlV9EJU')
uber_client = UberRidesClient(uber_session)


dx = 10000
dy = 10000

def GetUBerEstimate(lat_origin, long_origin):
    lat_dest = lat_origin + (180/math.pi)*(dy/6378137)
    long_dest = long_origin + (180/math.pi)*(dx/6378137)/math.cos(lat_origin)
    try:
        response = uber_client.get_price_estimates(
        start_latitude=lat_origin,
        start_longitude=long_origin,
        end_latitude=lat_dest,
        end_longitude=long_dest,
        seat_count=2
        )
    
        estimate = response.json.get('prices')
        high_estimate_uberx = estimate[1]['high_estimate']
        
    except IndexError:
        print("Oops!  That was no valid index")
        high_estimate_uberx = -1    
    except Exception as e:
        #print(e.message)
        high_estimate_uberx = -1
    return str(high_estimate_uberx)



eastern = timezone('US/Eastern')
utc = timezone('UTC')

fName = 'data/tweets_streamed_dataset.txt'
class MyListener(StreamListener):


    
    def on_status(self, status):
        if (status.geo != None):
                #print(status.geo)
                tweet_text=status.text
                tweet_id=jsonpickle.encode(status.id, unpicklable=False)
                created_at=jsonpickle.encode(status.created_at, unpicklable=False)
                utc_created_at = utc.localize(status.created_at).astimezone(eastern)
                tweet_text = jsonpickle.encode(status.text, unpicklable=False)
                tweet_coordinates= jsonpickle.encode(status.geo['coordinates'],unpicklable=False)
                lat_origin=(tweet_coordinates.split(",",1)[0].split("[",1)[1]).strip()
                long_origin=(tweet_coordinates.split(",",1)[1].split("]",1)[0]).strip()
                uber_estimate = GetUBerEstimate(float(lat_origin),float(long_origin))
                f.write(tweet_id + '\t' + str(utc_created_at) + '\t' + tweet_text +'\t' + tweet_coordinates + '\t'+lat_origin+'\t' +long_origin+ '\t' + uber_estimate + '\n')
                f.flush()
    def on_error(self, status):
        print(status)
        return True
 
twitter_stream = Stream(auth, MyListener())


#twitter_stream.filter(track=keyword_list, languages=['en'])
with open(fName, 'w',1) as f:
    f.write("tweet_id\ttweet_time\ttweet_text\ttweet_coordinates\tlat_origin\tlong_origin\tuber_estimate\n")
    twitter_stream.filter(locations=loc)

KeyboardInterrupt: 